In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Data

In [2]:
df = pd.read_csv('data_20180119_20180420.csv', index_col=0)
df.head()

,t0,t1,K1,pi1,t2,K2,pi2
0,2018-01-02,2018-01-19,100.0,23.936454,2018-04-20,80.0,49.401790
1,2018-01-02,2018-01-19,110.0,20.385049,2018-04-20,101.5,46.701200
2,2018-01-02,2018-01-19,115.0,18.609347,2018-04-20,102.5,46.575591
3,2018-01-02,2018-01-19,116.0,18.254206,2018-04-20,105.0,46.261569
4,2018-01-02,2018-01-19,117.0,17.899066,2018-04-20,108.0,45.884742


In [3]:
t0List = list(df.t0.unique())

# dimension: K[k-th t0][t1 and t2][i-th option with i-th strike]
K = [[np.array(df.loc[df.t0 == t0,'K1']),np.array(df.loc[df.t0==t0,'K2'])] for t0 in t0List]

# dimension: Pi[k-th t0][t1 and t2][i-th option with i-th price]
Pi = [[np.array(df.loc[df.t0 == t0,'pi1']),np.array(df.loc[df.t0==t0,'pi2'])] for t0 in t0List]

In [4]:
# number of t0
N_t0 = len(t0List)

# number of times (t1, t2)
T = 2

# number of options in hedging
N1, N2 = 20, 20
Size = max(N1, N2)

In [5]:
# Assume marginal distribution is uniform
def margDistr(batch_size):
    data = np.random.uniform(size=(batch_size, T))
    for t in range(T):
        data[:, t] = S_min + (S_max - S_min) * data[:, t] # just a simple test, need further consideration
    return data

In [6]:
# arbitrary
S_t0 = 143
S_max = 180
S_min = 100
BATCH_SIZE = 100

S = margDistr(BATCH_SIZE)

In [7]:
Phi = (S[:,1]-S[:,0])
# define the payoff function of the derivative

### Variables

In [8]:
# parameter: d
d = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, 1], dtype=tf.float64),
    trainable = True,
    name = 'd'
)

In [9]:
# parameter: lambda
Lambda = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, T, Size], dtype=tf.float64),
    trainable = True,
    name = 'lambda'
)

In [ ]:
# # parameter: Delta
# Delta = tf.Variable(
#     initial_value = tf.random.normal(shape=[N_t0, T-1], dtype=tf.float64),
#     trainable = True,
#     name = 'Delta'
# )

In [69]:
# neural network: Delta(S_1)
Delta = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(BATCH_SIZE,)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(N_t0, activation='softmax', dtype=tf.float64)
])

### The step-by-step computation of the objective function

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2}$

In [84]:
hedgePrice = []
for n in range(N_t0):
    p = d[n]
    for t in range(T):
        p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
    hedgePrice.append(p)

In [85]:
# take a look
hedgePrice

[<tf.Tensor: shape=(1,), dtype=float64, numpy=array([-38.14193511])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-861.12518303])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-4160.23171665])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-350.11579889])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-5512.05424333])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-552.33866013])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-9143.87783473])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-5342.98305873])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-401.47841593])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-5649.3146934])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-1541.42326033])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-4123.77455746])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([-5286.36934893])>]

$d+\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^++\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^++\Delta_{1}(S_1)\cdot(S_2-S_1)$

In [82]:
hedgeTerm = []
for n in range(N_t0):
    p = d[n]
    for t in range(T):
        p += tf.reduce_sum(Lambda[n,t,:]*np.maximum(S[:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
#     for t in range(T-1):
#         p += Delta[n,t]*(S[:,t+1]-S[:,t])
    p += Delta(S[:,0].reshape((1,-1)))[0,n]*(S[:,1]-S[:,0])
    hedgeTerm.append(p)

In [83]:
# take a look
hedgeTerm

[<tf.Tensor: shape=(100,), dtype=float64, numpy=
 array([ 41.9472133 , -48.59596467,  23.11110981,  37.42765881,
        -30.32365856,  62.12032243,  21.26237948,  58.72372108,
         26.66639615,   8.5088482 ,  77.99252654, -45.03010786,
         66.32159102,  62.00999275,   4.53443259,  49.03752039,
         42.61412097,  -5.56648333,  32.31547344,  17.54611289,
        -16.70317302,  41.77093713,  25.39761771, -19.50916536,
         -6.6706892 ,   1.92811476,  60.47204767,  84.64269149,
        -24.7748149 ,  -6.95496875,  36.81189775,  29.06241293,
         55.1742134 ,  22.02151114,  48.47675265, -27.39104469,
         62.35204867,  23.95770302,  72.28340478,  -7.4581648 ,
         72.43962686,  19.81459016,  18.52743023,  11.90260829,
         66.85483764, -45.2779009 ,  64.97424218,  46.26438085,
         44.16079571,  21.10006705,  73.0154536 ,  59.90535822,
         23.68970929,  25.85321888, -16.02045238,  67.32865744,
         26.61195402,  17.99075741,  70.09467142,   9.3

$\inf_{h\in\mathcal{H}^m}\int hd\mu_0+\int \beta_\gamma(c-h)d\theta$

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2} + \Gamma\cdot\left[\left(\Phi(S_1,S_2)-d-\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^+-\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^+-\Delta_{1}(S_1)\cdot(S_2-S_1)\right)^+\right]^2$

In [15]:
# objective function
Price = tf.reduce_mean(hedgePrice, axis=1)

Gamma = 100
diff = Phi - tf.stack(hedgeTerm, axis=0)
Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)

ObjFunc = Price + Gamma*Penalty

In [16]:
# take a look
Price

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([ 890.64463977,  277.64992917,  401.84222244, -622.03040404,
       -215.90788441, -912.94438297,  806.74059786,  871.94900119,
        283.79872458, -935.51027626, -503.46666635, -229.43346824,
       -497.44295917])>

In [17]:
# take a look
Penalty

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([0.00000000e+00, 6.71164304e+02, 5.03725794e+01, 4.24266390e+04,
       2.03314691e+04, 7.86120898e+04, 0.00000000e+00, 0.00000000e+00,
       2.56010303e+01, 9.21620482e+04, 1.12366541e+04, 5.28041552e+04,
       6.22570246e+04])>

In [18]:
# take a look
ObjFunc

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([8.90644640e+02, 6.73940803e+04, 5.43910016e+03, 4.24204187e+06,
       2.03293100e+06, 7.86029604e+06, 8.06740598e+02, 8.71949001e+02,
       2.84390175e+03, 9.21526931e+06, 1.12316194e+06, 5.28018609e+06,
       6.22520502e+06])>

### Optimisation

In [86]:
# Combine all the computation as a function
def ObjFunc(d, Lambda, Delta, K, Pi, Phi, S):
    hedgePrice = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
        hedgePrice.append(p)
    
    hedgeTerm = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*np.maximum(S[:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
#         for t in range(T-1):
#             p += Delta[n,t]*(S[:,t+1]-S[:,t])
        p += Delta(S[:,0].reshape((1,-1)))[0,n]*(S[:,1]-S[:,0]) # Delta as a neural network
        hedgeTerm.append(p)
    
    Price = tf.reduce_mean(hedgePrice, axis=1)
    Gamma = 100
    diff = Phi-tf.stack(hedgeTerm, axis=0)
    Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)
    
    return Price + Gamma*Penalty, Price

In [87]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = 0.01)

In [101]:
# for step in range(10000):
#     with tf.GradientTape() as tape:
#         obj, price = ObjFunc(d, Lambda, Delta, K, Pi, Phi, S)
#     gradients = tape.gradient(obj, [d, Lambda, Delta])
#     optimizer.apply_gradients(zip(gradients, [d, Lambda, Delta]))

In [94]:
for step in range(10000):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(d)
        tape.watch(Lambda)
        for variable in Delta.trainable_variables:
            tape.watch(variable)

        obj, price = ObjFunc(d, Lambda, Delta, K, Pi, Phi, S)
    
    gradients_d, gradients_Lambda = tape.gradient(obj, [d, Lambda])
    gradients_Delta = tape.gradient(obj, Delta.trainable_variables)
    optimizer.apply_gradients(
        zip([gradients_d, gradients_Lambda] + gradients_Delta, [d, Lambda] + Delta.trainable_variables)
    )
    
    del tape

In [95]:
price

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([ 3.59567264e+00, -8.80870327e+02, -4.11492280e+03, -3.50133134e+02,
       -5.47860419e+03, -5.45572981e+02, -9.13827258e+03, -5.32238297e+03,
       -4.20565520e+02, -5.68645299e+03, -1.57123201e+03, -4.13621173e+03,
       -5.30807336e+03])>

In [96]:
obj

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([ 3.59567264e+00, -8.80870327e+02, -4.11492280e+03, -3.50133134e+02,
       -5.47860419e+03, -5.45572981e+02, -9.13827258e+03, -5.32238297e+03,
       -4.20565520e+02, -5.68645299e+03, -1.57123201e+03, -4.13621173e+03,
       -5.30807336e+03])>